In [ ]:
# 从合并稳定的网络，识别重要节点，设计资源分配策略
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.lines import Line2D

# 读取数据
node_file = r'c:\Users\Administrator\Desktop\network_generate\network_generate\T500(11)\nodes.csv'
edge_file = r'c:\Users\Administrator\Desktop\network_generate\network_generate\T500(11)\edges.csv'

edges = pd.read_csv(edge_file)
nodes = pd.read_csv(node_file)
print(f"节点总数: {nodes.shape[0]}\n")

G = nx.from_pandas_edgelist(edges, 'Source', 'Target', edge_attr=True, create_using=nx.DiGraph())
G.add_nodes_from(nodes['Node'].tolist())

for _, row in nodes.iterrows():
    for col in nodes.columns:
        G.nodes[row['Node']][col] = row[col]

grades = nodes.set_index('Node')['grade'].to_dict()

# 修改后的指标名称
metrics = [
    ('Average degree', lambda G: dict(G.degree())),
    ('In-degree', lambda G: dict(G.in_degree())),
    ('Weighted degree', lambda G: dict(G.degree(weight='weight'))),
    ('Weighted in-degree', lambda G: dict(G.in_degree(weight='weight'))),
    ('PageRank', lambda G: nx.pagerank(G)),
    ('Lost patients(LP)', lambda G: {n: G.nodes[n]['LP'] for n in G.nodes()}),
    ('Out-degree', lambda G: dict(G.out_degree())),
    ('Weighted out-degree', lambda G: dict(G.out_degree(weight='weight')))
]

def get_top_5_percent(G, func, name):
    data = func(G)
    df = pd.DataFrame(data.items(), columns=['Node', name])
    top_k = max(1, int(len(df) * 0.05))
    print(f"指标 [{name}] 的节点总数: {len(df)}，Top 5% 数量: {top_k}\n")
    return df.nlargest(top_k, name).reset_index(drop=True)

top_nodes = {name: get_top_5_percent(G, func, name) for name, func in metrics}

GRADE_COLORS = {1: '#D2CBB0', 2: '#ff7f0e', 3: '#D80032'}

fig, ax = plt.subplots(figsize=(22, 14))
node_size = 3000
row_gap = 0.6
node_pos = {}

max_len = max(len(df) for df in top_nodes.values())
x_labels = list(range(1, max_len + 1))

def compute_radius_offset(ax, size):
    pixel_radius = np.sqrt(size / np.pi)
    x0, y0 = ax.transData.transform((0, 0))
    x1, y1 = ax.transData.transform((0, 1))
    return pixel_radius / abs(y1 - y0)

radius_offset = compute_radius_offset(ax, node_size)

for i, (metric, df) in enumerate(top_nodes.items()):
    y = (len(metrics) - 1 - i) * row_gap
    ax.text(0.2, y, metric, fontsize=20, va='center', ha='right', weight='bold')
    for j, node in enumerate(df['Node']):
        node_id = int(node)
        x = j + 1
        color = GRADE_COLORS.get(grades.get(node, 1), '#D2CBB0')
        ax.scatter(x, y, s=node_size, color=color, edgecolors='black', linewidths=1.2, zorder=3)
        ax.text(x, y, str(node_id), fontsize=20, ha='center', va='center', weight='bold', zorder=4, color='white')
        node_pos.setdefault(node_id, []).append((x, y, i))

# 连线
for node, positions in node_pos.items():
    positions = sorted(positions, key=lambda x: x[2])
    for k in range(len(positions) - 1):
        (x1, y1, idx1), (x2, y2, idx2) = positions[k], positions[k + 1]
        if abs(idx2 - idx1) == 1:
            ax.plot([x1, x2], [y1 - radius_offset, y2 + radius_offset],
                    color='blue', linewidth=5, alpha=0.5, zorder=2)

# 图例
legend_elements = [
    Line2D([0], [0], marker='o', color='w', label='Grade 1',
           markerfacecolor=GRADE_COLORS[1], markersize=15),
    Line2D([0], [0], marker='o', color='w', label='Grade 2',
           markerfacecolor=GRADE_COLORS[2], markersize=15),
    Line2D([0], [0], marker='o', color='w', label='Grade 3',
           markerfacecolor=GRADE_COLORS[3], markersize=15),
]
ax.legend(handles=legend_elements, loc='upper center',
          ncol=3, fontsize=20, frameon=False)

# 坐标轴设置
ax.set_xticks(x_labels)
ax.set_xticklabels([str(i) for i in x_labels], fontsize=20)
ax.set_yticks([])
ax.set_xlim(0.5, max_len + 0.5)
ax.set_ylim(-0.5, len(metrics) * row_gap)
ax.set_facecolor('white')
ax.tick_params(axis='x', labelsize=20)

plt.tight_layout()
output_pdf = r'c:\Users\Administrator\Desktop\network_generate1\plots\main_nodes.pdf'
plt.savefig(output_pdf, dpi=300, bbox_inches='tight', format='pdf')
print(f"图表已保存为 PDF 格式: {output_pdf}")
plt.show()
plt.show()


In [ ]:
# 使用Louvain算法,社区识别 
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull
from scipy.interpolate import splprep, splev
from networkx.algorithms import community as nx_comm
from matplotlib import rcParams
from matplotlib.patches import Polygon, Circle
from matplotlib.path import Path
from matplotlib.patches import PathPatch
from scipy.spatial import ConvexHull
from matplotlib.lines import Line2D
# 设置中文字体显示
plt.rcParams['font.sans-serif'] = ['SimHei']  # 使用黑体显示中文
plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题
def smooth_polygon(points, closed=True, smooth_factor=0.2):
    """生成平滑曲线包围轮廓"""
    from scipy.interpolate import splprep, splev
    points = np.array(points)
    if closed:
        points = np.concatenate([points, [points[0]]])
    tck, u = splprep([points[:, 0], points[:, 1]], s=0, per=closed)
    unew = np.linspace(0, 1.0, 100)
    out = splev(unew, tck)
    return Polygon(np.vstack(out).T, closed=True)

def detect_and_visualize_communities(G):
    

    G_undirected = G.to_undirected()
    raw_communities = nx_comm.louvain_communities(G_undirected, resolution=1, seed=123)

    # 获取位置数据
    pos = {}
    for node in G.nodes():
        try:
            loc = G.nodes[node].get('location', None)
            if loc and not pd.isna(loc):
                if isinstance(loc, str):
                    loc = eval(loc)
                pos[node] = (float(loc[0]), float(loc[1]))
            else:
                pos[node] = None
        except:
            pos[node] = None
    if not pos or any(v is None for v in pos.values()):
        pos = nx.spring_layout(G_undirected, seed=42)

    # 拆分大社区与小社区
    large_communities = [c for c in raw_communities if len(c) >= 5]
    small_communities = [c for c in raw_communities if len(c) < 5]
    final_communities = [set(comm) for comm in large_communities]

    # 小社区节点重新分配
    for small_comm in small_communities:
        for node in small_comm:
            node_pos = np.array(pos[node])
            min_dist = float('inf')
            best_comm_idx = -1
            for idx, large_comm in enumerate(final_communities):
                large_pos = np.array([pos[n] for n in large_comm])
                center = large_pos.mean(axis=0)
                dist = np.linalg.norm(node_pos - center)
                if dist < min_dist:
                    min_dist = dist
                    best_comm_idx = idx
            final_communities[best_comm_idx].add(node)

    partition = {node: i for i, comm in enumerate(final_communities) for node in comm}

    fig, ax = plt.subplots(figsize=(12, 10))
    community_colors = plt.cm.tab20(np.linspace(0, 1, len(final_communities)))

    for i, comm in enumerate(final_communities):
        comm_points = np.array([pos[node] for node in comm])
        center = comm_points.mean(axis=0)

        try:
            if len(comm_points) >= 3:
                hull = ConvexHull(comm_points)
                hull_pts = comm_points[hull.vertices]
                # 闭合曲线首尾连接
                hull_pts = np.concatenate([hull_pts, hull_pts[:1]], axis=0)
                # 样条拟合
                tck, _ = splprep([hull_pts[:, 0], hull_pts[:, 1]], s=0, per=True)
                smooth = splev(np.linspace(0, 1, 200), tck)
                ax.fill(smooth[0], smooth[1],
                        facecolor=community_colors[i],
                        edgecolor=community_colors[i],
                        linewidth=2,
                        alpha=0.15)
                ax.plot(smooth[0], smooth[1],
                        color=community_colors[i],
                        linewidth=1.5)
            else:
                radius = 3 * max(np.linalg.norm(comm_points - center, axis=1)) if len(comm_points) > 1 else 1.0
                circle = Circle(center, radius=radius,
                                facecolor=community_colors[i],
                                edgecolor='black',
                                linewidth=2,
                                alpha=0.15)
                ax.add_patch(circle)
        except Exception as e:
            print(f"社区 {i} 绘制错误: {e}")
            continue

        # 添加社区编号
        ax.text(center[0], center[1], f"community {i+1}",
                fontsize=20, fontweight='bold',
                color='black',
                ha='center', va='center',
                bbox=dict(boxstyle="round,pad=0.2", facecolor='white', edgecolor='none', alpha=0.7))

    # 绘制边
    nx.draw_networkx_edges(
        G_undirected, pos,
        alpha=0.15,
        edge_color='gray',
        width=0.8,
        ax=ax
    )

    # 按等级绘制节点
    grade_colors = {
        'grade = 1': '#808080',
        'grade = 2': '#FFD700',
        'grade = 3': '#FF4500'
    }
    
    for grade, color in grade_colors.items():
        nodes = [node for node in G.nodes() if G.nodes[node].get('grade') == grade]
        nx.draw_networkx_nodes(
            G_undirected, pos,
            nodelist=nodes,
            node_size=80,
            node_color=color,
            edgecolors='white',
            linewidths=0.5,
            label=f'{grade}',
            ax=ax
        )

    legend_elements = [
    Line2D([0], [0], marker='o', color='w', label='Grade 1', 
           markerfacecolor=grade_colors['grade = 1'], markersize=15),
    Line2D([0], [0], marker='o', color='w', label='Grade 2', 
           markerfacecolor=grade_colors['grade = 2'], markersize=15),
    Line2D([0], [0], marker='o', color='w', label='Grade 3', 
           markerfacecolor=grade_colors['grade = 3'], markersize=15),
]

    ax.legend(
        handles=legend_elements,
        loc='lower center',            # 图下方居中
        bbox_to_anchor=(0.5, 0.00),   # 控制距离图底部的偏移
        ncol=3,                        # 横向排列为3列
        fontsize=20,
        frameon=True
    )
    #ax.set_title(f"Community structure of referral network（共 {len(final_communities)} 个社区）", pad=20)

    if any('location' in G.nodes[node] for node in G.nodes()):
        ax.set_xlabel("longitude", fontsize=20)
        ax.set_ylabel("latitude", fontsize=20)

    plt.tight_layout()
    plt.savefig(r'c:\Users\Administrator\Desktop\network_generate1\plots\Community structure.pdf', dpi=300, bbox_inches='tight')
    plt.show()
    return partition


if __name__ == '__main__':
    node_file = r'c:\Users\Administrator\Desktop\network_generate1\original_data\nodes(cure).csv'
    edge_file = r'c:\Users\Administrator\Desktop\network_generate1\T500\T500(11)\edges.csv'
    nodes_df = pd.read_csv(node_file)
    edges_df = pd.read_csv(edge_file)

    grade_map = {'1': 'grade = 1', '2': 'grade = 2', '3': 'grade = 3'}
    G = nx.DiGraph()
    for _, row in nodes_df.iterrows():
        raw_grade = str(row.get('grade')).strip()
        grade = grade_map.get(raw_grade, raw_grade)
        location = row.get('location')
        G.add_node(row['Node'], grade=grade, location=location)

    for _, row in edges_df.iterrows():
        G.add_edge(row['Source'], row['Target'], weight=row['weight'])

    partition = detect_and_visualize_communities(G)
  
